In [0]:
from pymongo import MongoClient
from datetime import datetime
import pandas as pd
import pytz

# Connect to MongoDB
def connect_to_mongo(uri, db_name, collection_name):
    """
    Connect to MongoDB and return a collection object.
    """
    client = MongoClient(uri)
    db = client[db_name]
    collection = db[collection_name]
    print(f"Connected to MongoDB collection: {collection_name}")
    return collection

# Fetch all data from MongoDB
def fetch_all_data_from_mongo(collection):
    """
    Fetch all historical data from MongoDB up to the current date.
    """

    # Query all documents
    cursor = collection.find()
    data = pd.DataFrame(list(cursor))
    
    # Convert the 'Date' field from string to datetime
    data['Date'] = pd.to_datetime(data['Date'])
    
    # Filter rows where the 'Date' is before the current date
    filtered_data = data[data['Date'] < datetime.now()]
    return filtered_data

def fetch_current_date_data(collection):
    """
    Fetch only the data for the current date from MongoDB.

    Parameters:
    - collection: MongoDB collection object

    Returns:
    - current_date_data: Pandas DataFrame containing data for the current date
    """
    # Query all documents
    cursor = collection.find()
    data = pd.DataFrame(list(cursor))
    
    # Safeguard: Check if the 'Date' column exists
    if 'Date' in data.columns:
        # Convert the 'Date' field from string to datetime
        data['Date'] = pd.to_datetime(data['Date'], errors='coerce')  # Handle invalid dates gracefully

        # Get today's date in YYYY-MM-DD format
        today = datetime.now().date()

        # Filter rows where the 'Date' matches today's date
        current_date_data = data[data['Date'].dt.date == today]
    else:
        print("Warning: 'Date' column not found in the fetched data.")
        current_date_data = pd.DataFrame()  # Return an empty DataFrame if 'Date' column is missing

    return current_date_data

def insert_dataframe_to_mongo(collection, dataframe):
    data_dict = dataframe.to_dict("records")  # Convert DataFrame to list of dictionaries
    collection.insert_many(data_dict)        # Insert into MongoDB

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
File <command-4215439981543840>:1
----> 1 from pymongo import MongoClient
      2 import pandas as pd
      4 # Connect to MongoDB

File /databricks/python_shell/dbruntime/PythonPackageImportsInstrumentation/__init__.py:171, in _create_import_patch.<locals>.import_patch(name, globals, locals, fromlist, level)
    166 thread_local._nest_level += 1
    168 try:
    169     # Import the desired module. If you’re seeing this while debugging a failed import,
    170     # look at preceding stack frames for relevant error information.
--> 171     original_result = python_builtin_import(name, globals, locals, fromlist, level)
    173     is_root_import = thread_local._nest_level == 1
    174     # `level` represents the number of leading dots in a relative import statement.
    175     # If it's zero, then this is an absolute import.

ModuleNo